# Complete Markowitz Analysis
This will complete the results under the following:

| n=4 returns ~ N(\mu,\sigma)  | n=4 returns ~ N(\mu, sigma) 25% correlation  | n=4 returns ~ N(\mu, sigma) 50% correlation  | n=4 returns ~ N(\mu, sigma) 75% correlation  |
|------------------------------|----------------------------------------------|----------------------------------------------|----------------------------------------------|
| n=8 returns ~ N(\mu,\sigma)  | n=8 returns ~ N(\mu, sigma) 25% correlation  | n=8 returns ~ N(\mu, sigma) 50% correlation  | n=8 returns ~ N(\mu, sigma) 75% correlation  |
| n=16 returns ~ N(\mu,\sigma) | n=16 returns ~ N(\mu, sigma) 25% correlation | n=16 returns ~ N(\mu, sigma) 50% correlation | n=16 returns ~ N(\mu, sigma) 75% correlation |

using the method laid out in 4.1, Markowitz portfolio optimization

## Read Packages

In [272]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import cvxopt as opt
from cvxopt import blas, solvers
import pandas as pd
from pylab import rcParams
from mosek import iparam
import time
solvers.options['show_progress'] = False

In [2]:
import plotly as py
import plotly.tools as tls
from plotly.graph_objs import *

## Simulate Data
Normal distribution with 0% correlation

In [200]:
## Set number of assets
n_assets = 4

## Set number of observations
n_obs = 1000

return_vec = np.random.randn(n_assets, n_obs)
return_vec.shape

(4, 1000)

Create function to generate random weights

In [4]:
def rand_weights(n):
    '''Produces n random weights that sum to 1'''
    k = np.random.rand(n)
    return k / sum(k)

Create function to generate random portfolio

In [5]:
def random_portfolio(returns):
    '''
    Returns the mean and standard deviation of returns for a randomly generated portfolio
    '''
    
    p = np.asmatrix(np.mean(returns, axis=1))
    w = np.asmatrix(rand_weights(returns.shape[0]))
    C = np.asmatrix(np.cov(returns))
    
    mu = w * p.T
    sigma = np.sqrt(w * C * w.T)
    
    # Test for outliers to make plot prettier
    if sigma > 2:
        return random_portfolio(returns)
    return mu, sigma

Generates $n$ random portfolios

In [6]:
n_portfolios = 500
means, stds = np.column_stack([
    random_portfolio(return_vec)
    for _ in range(n_portfolios)
])

## Optimization

Create a series of functions that perform optimizations

In [264]:
def optimal_portfolio(returns):
    n = len(returns)
    returns = np.asmatrix(returns + 5)
    
    N = n_obs
    mus = [10**(5.0 * t/N - 1.0) for t in range(N)]
    
    # Convert to cvxopt matrices
    S = opt.matrix(np.cov(returns))
    pbar = opt.matrix(np.mean(returns, axis = 1))
    
    # Create constraint matrices
    G = -opt.matrix(np.eye(n)) # negative nxn identity matrix
    h = opt.matrix(0.0, (n,1))
    A = opt.matrix(1.0, (1,n))
    b = opt.matrix(1.0)
    
    # Calculate efficient frontier weights using quadratic programming
    portfolios = [solvers.qp(mu*S, -pbar, G, h, A, b)['x']
                  for mu in mus]
    ## Calculate risk and returns for frontier
    returns = [blas.dot(pbar, x) for x in portfolios]
    risks = [np.sqrt(blas.dot(x, S*x)) for x in portfolios]
    ## Calculate the 2nd degree polynomail of the frontier curve
    m1 = np.polyfit(returns, risks, 2)
    x1 = np.sqrt(m1[2] / m1[0])
    ## Calculate the optimal portfolio
    wt = solvers.qp(opt.matrix(x1 * S), -pbar, G, h, A, b)['x']
    return np.asarray(wt), returns, risks

Create function that performs 1 optimization

In [270]:
def one_optimization(n_assets, n_obs):
    '''
    First, simulates portfolios then optimizes. 
    This does 30 replications
    '''
    weight_res = np.zeros((99,n_assets))
    return_res = np.zeros((99,n_obs))
    risks_res = np.zeros((99,n_obs))
    
    for i in range(99):
        n_portfolios = 500
        np.random.seed(i)
        return_vec = np.random.randn(n_assets, n_obs)# need to set the seeds here so others use the same values
        means, stds = np.column_stack([
            random_portfolio(return_vec)
            for _ in range(n_portfolios)
            ])
        weights, returns, risks = optimal_portfolio(return_vec)
        weight_res[i,:] = weights.T
        return_res[i,:] = np.array(returns).T
        risks_res[i,:] = np.array(risks).T
    return weight_res, return_res, risks_res

In [ ]:
def one_optimization_zeros(n_assets, n_obs):
    '''
    This is exactly like one_optimization except it replaces 75% of asset 1 and 2 with zeros
    First, simulates portfolios then optimizes. 
    This does 30 replications
    '''
    weight_res = np.zeros((99,n_assets))
    return_res = np.zeros((99,n_obs))
    risks_res = np.zeros((99,n_obs))
    
    for i in range(99):
        n_portfolios = 500
        np.random.seed(i)
        return_vec = np.random.randn(n_assets, n_obs)# need to set the seeds here so others use the same values
        b = [0 if np.random.randint(0, len(return_vec[2])) < 2 else i for i in return_vec[2]]
        np.random.seed(75)
        c = [0 if np.random.randint(0, len(return_vec[2])) < 2 else i for i in return_vec[2]]
        return_vec[1,:] = b
        return_vec[2,:] = c
        means, stds = np.column_stack([
            random_portfolio(return_vec)
            for _ in range(n_portfolios)
            ])
        weights, returns, risks = optimal_portfolio(return_vec)
        weight_res[i,:] = weights.T
        return_res[i,:] = np.array(returns).T
        risks_res[i,:] = np.array(risks).T
    return weight_res, return_res, risks_res

## Perform the optimizations specified
## $n = 4$ 

In [274]:
start = time.time()
n4_normal_weight, n4_normal_return, n4_normal_risks = one_optimization(4, 1000)
end = time.time()
print(start - end)

-62.01717400550842


In [309]:
np.round(n4_normal_weight.mean(axis=0),3)

array([0.251, 0.25 , 0.249, 0.251])

In [310]:
n4_normal_weight_os, n4_normal_return_os, n4_normal_risks_os = one_optimization_zeros(4, 1000)

In [311]:
np.round(n4_normal_weight_os.mean(axis=0),3)

array([0.335, 0.132, 0.204, 0.329])

## $n = 8$ 

In [312]:
n8_normal_weight, n4_normal_return, n4_normal_risks = one_optimization(8, 1000)

In [313]:
np.round(n8_normal_weight.mean(axis=0),3)

array([0.126, 0.126, 0.125, 0.125, 0.126, 0.125, 0.125, 0.123])

In [314]:
n8_normal_weight_os, n4_normal_return_os, n4_normal_risks_os = one_optimization_zeros(8, 1000)

In [315]:
np.round(n8_normal_weight_os.mean(axis=0),3)

array([0.146, 0.073, 0.071, 0.141, 0.144, 0.142, 0.144, 0.14 ])

## $n=16$

In [316]:
n16_normal_weight, n4_normal_return, n4_normal_risks = one_optimization(16, 1000)

In [317]:
np.round(n16_normal_weight.mean(axis=0), 3)

array([0.063, 0.063, 0.063, 0.063, 0.063, 0.063, 0.064, 0.061, 0.062,
       0.064, 0.061, 0.061, 0.064, 0.063, 0.062, 0.062])

In [319]:
n16_normal_weight_os, n16_normal_return_os, n16_normal_risks_os = one_optimization_zeros(16,1000)

In [320]:
np.round(n16_normal_weight_os.mean(axis=0),3)

array([0.067, 0.038, 0.028, 0.063, 0.067, 0.066, 0.069, 0.063, 0.069,
       0.07 , 0.068, 0.064, 0.065, 0.069, 0.068, 0.064])

## Play with Correlation

## Functions

In [288]:
def cor_optimization(n_assets, n_obs, r):
    '''
    First, simulates portfolios then optimizes. 
    This does 30 replications
    n_assets: number of assets
    n_obs: number of observations
    r: covariance matrix
    '''
    weight_res = np.zeros((99,n_assets))
    return_res = np.zeros((99,n_obs))
    risks_res = np.zeros((99,n_obs))
    
    for i in range(99):
        n_portfolios = 500
        np.random.seed(i)
        
        rng = np.random.default_rng()
        return_vec = rng.multivariate_normal(np.zeros(n_assets), cov = r, size = n_obs).T
        
        means, stds = np.column_stack([
            random_portfolio(return_vec)
            for _ in range(n_portfolios)
            ])
        weights, returns, risks = optimal_portfolio(return_vec)
        weight_res[i,:] = weights.T
        return_res[i,:] = np.array(returns).T
        risks_res[i,:] = np.array(risks).T
    return weight_res, return_res, risks_res

In [ ]:
def cor_optimization_zeros(n_assets, n_obs, r):
    '''
    First, simulates portfolios then optimizes. 
    This does 30 replications
    n_assets: number of assets
    n_obs: number of observations
    r: covariance matrix
    '''
    weight_res = np.zeros((99,n_assets))
    return_res = np.zeros((99,n_obs))
    risks_res = np.zeros((99,n_obs))
    
    for i in range(99):
        n_portfolios = 500
        np.random.seed(i)
        
        rng = np.random.default_rng()
        return_vec = rng.multivariate_normal(np.zeros(n_assets), cov = r, size = n_obs).T
        b = [0 if np.random.randint(0, len(return_vec[2])) < 2 else i for i in return_vec[2]]
        np.random.seed(75)
        c = [0 if np.random.randint(0, len(return_vec[2])) < 2 else i for i in return_vec[2]]
        return_vec[1,:] = b
        return_vec[2,:] = c
        
        means, stds = np.column_stack([
            random_portfolio(return_vec)
            for _ in range(n_portfolios)
            ])
        weights, returns, risks = optimal_portfolio(return_vec)
        weight_res[i,:] = weights.T
        return_res[i,:] = np.array(returns).T
        risks_res[i,:] = np.array(risks).T
    return weight_res, return_res, risks_res

## Results for Correlation between 0.8-0.9

## $n=4$

In [ ]:
n = 4
r = np.random.uniform(0.8, 0.9, (n,n))
np.fill_diagonal(r, 1)

In [283]:
n4_corr_weight, n4_corr_return, n4_corr_risks = cor_optimization(4, 1000, r)

<ipython-input-281-4a1b8cf2d84f>:18: RuntimeWarning:

covariance is not positive-semidefinite.



In [284]:
np.round(n4_corr_weight.mean(axis=0),3)

array([0.181, 0.275, 0.276, 0.269])

In [290]:
n4_corr_weight_os, n4_smallmvn_return_os, n4_smallmvn_risks_os = cor_optimization_zeros(4, 1000, r)

<ipython-input-289-eec99e35fddc>:18: RuntimeWarning:

covariance is not positive-semidefinite.



In [292]:
np.round(n4_corr_weight_os.mean(axis = 0), 3)

array([0.298, 0.165, 0.213, 0.324])

## $n = 8$

In [294]:
n = 8
r = np.random.uniform(0.8, 0.9, (n,n))
np.fill_diagonal(r, 1)

In [295]:
n8_corr_weight, n8_corr_return, n8_corr_risks = cor_optimization(8, 1000, r)

<ipython-input-288-4a1b8cf2d84f>:18: RuntimeWarning:

covariance is not positive-semidefinite.



zeros

In [296]:
np.round(n8_corr_weight.mean(axis = 0), 3)

array([0.14 , 0.201, 0.106, 0.002, 0.097, 0.174, 0.158, 0.121])

In [297]:
n8_corr_weight_os, n8_smallmvn_return_os, n8_smallmvn_risks_os = cor_optimization_zeros(8, 1000, r)

<ipython-input-289-eec99e35fddc>:18: RuntimeWarning:

covariance is not positive-semidefinite.



In [300]:
np.round(n8_corr_weight_os.mean(axis = 0),3)

array([0.128, 0.072, 0.104, 0.007, 0.151, 0.191, 0.188, 0.16 ])

## $n=16$

In [301]:
n = 16
r = np.random.uniform(0.8, 0.9, (n,n))
np.fill_diagonal(r, 1)

In [303]:
n16_corr_weight, n16_corr_return, n16__corr_risks = cor_optimization(16,1000,r)

<ipython-input-288-4a1b8cf2d84f>:18: RuntimeWarning:

covariance is not positive-semidefinite.



In [304]:
np.round(n16_corr_weight.mean(axis=0),3)

array([0.04 , 0.073, 0.104, 0.064, 0.027, 0.023, 0.059, 0.079, 0.071,
       0.022, 0.147, 0.061, 0.076, 0.047, 0.066, 0.042])

In [305]:
n16_corr_weight_os, n16_smallmvn_return_os, n16_smallmvn_risks_os = cor_optimization_zeros(16, 1000, r)

<ipython-input-289-eec99e35fddc>:18: RuntimeWarning:

covariance is not positive-semidefinite.



In [307]:
np.round(n16_corr_weight_os.mean(axis = 0), 3)

array([0.039, 0.045, 0.077, 0.062, 0.037, 0.03 , 0.086, 0.082, 0.075,
       0.023, 0.146, 0.051, 0.07 , 0.072, 0.064, 0.041])

## Results for Correlation between -0.6 and -0.5

## $n=4$

In [327]:
n = 4
r = np.random.uniform(-0.8, -0.5, (n,n))
np.fill_diagonal(r, 1)

In [328]:
n4_neg_weight, n4_neg_return, n4_neg_risks = cor_optimization(4, 1000, r)

<ipython-input-288-4a1b8cf2d84f>:18: RuntimeWarning:

covariance is not positive-semidefinite.



In [329]:
np.round(n4_neg_weight.mean(axis = 0),3)

array([0.266, 0.245, 0.239, 0.25 ])

In [330]:
n4_neg_weight_os, n4_neg_return, n4_neg_risks = cor_optimization_zeros(4, 1000, r)

<ipython-input-289-eec99e35fddc>:18: RuntimeWarning:

covariance is not positive-semidefinite.



In [331]:
np.round(n4_neg_weight_os.mean(axis = 0), 3)

array([0.335, 0.165, 0.166, 0.333])

## $n=8$

In [337]:
n = 8
r = np.random.uniform(-0.8, -0.5, (n,n))
np.fill_diagonal(r, 1)

In [333]:
n8_neg_weight, n8_neg_return, n8_neg_risks = cor_optimization(8, 1000, r)

<ipython-input-288-4a1b8cf2d84f>:18: RuntimeWarning:

covariance is not positive-semidefinite.



In [334]:
np.round(n8_neg_weight.mean(axis=0), 3)

array([0.114, 0.101, 0.166, 0.114, 0.133, 0.118, 0.124, 0.129])

In [338]:
n8_neg_weight_os, n8_neg_return, n8_neg_risks = cor_optimization_zeros(8, 1000, r)

<ipython-input-289-eec99e35fddc>:18: RuntimeWarning:

covariance is not positive-semidefinite.



In [339]:
np.round(n8_neg_weight_os.mean(axis=0),3)

array([0.17 , 0.086, 0.086, 0.135, 0.146, 0.129, 0.131, 0.117])

## $n=16$

In [346]:
n = 16
r = np.random.uniform(-0.8, -0.5, (n,n))
np.fill_diagonal(r, 1)

In [341]:
n16_neg_weight, n16_neg_return, n16_neg_risks = cor_optimization(16, 1000, r)

<ipython-input-288-4a1b8cf2d84f>:18: RuntimeWarning:

covariance is not positive-semidefinite.



In [342]:
np.round(n16_neg_weight.mean(axis=0), 3)

array([0.069, 0.062, 0.054, 0.064, 0.072, 0.077, 0.065, 0.059, 0.059,
       0.076, 0.042, 0.054, 0.061, 0.059, 0.061, 0.066])

In [343]:
n16_neg_weight_os, n16_neg_return, n16_neg_risks = cor_optimization_zeros(16, 1000, r)

<ipython-input-289-eec99e35fddc>:18: RuntimeWarning:

covariance is not positive-semidefinite.



In [344]:
np.round(n16_neg_weight_os.mean(axis=0), 3)

array([0.073, 0.025, 0.033, 0.068, 0.075, 0.083, 0.069, 0.063, 0.062,
       0.081, 0.048, 0.06 , 0.057, 0.065, 0.063, 0.074])